In [13]:
from dotenv import load_dotenv
import os
from pymongo import MongoClient
import pandas as pd
import functions.utils as utils

In [2]:
load_dotenv()
MONGODB_URI = os.getenv("MONGODB_URI")

In [5]:
mongoClient = MongoClient(MONGODB_URI)
db = mongoClient["news_data"]
ieColl = db["ireland_news"]

## The Ireland News Dataset

In [28]:
# https://towardsdatascience.com/list-comprehensions-vs-for-loops-it-is-not-what-you-think-34071d4d8207
# Using array computation is a lot faster than using for loops and appending to a list
irelandNews = list(ieColl.find({}))
print(len(irelandNews))

irelandNewsDF = pd.DataFrame(irelandNews)
irelandNewsDF.head()

505


,_id,id,title,publish_date,source_country,sentiment
0,646fddfd0576b45037ddb2c6,111013564,Kilkenny-based recruitment company wins two in...,2023-04-18 08:59:00,ie,0.544
1,646fddfd0576b45037ddb2c7,88450232,Ministers Harris and Ryan welcome record numbe...,2023-01-17 21:41:06,ie,0.341
2,646fddfd0576b45037ddb2c8,75989338,New €4.8m state-of-the-art facility planned fo...,2022-12-05 11:06:50,ie,0.377
3,646fddfd0576b45037ddb2c9,92473892,‘Not possible’ to make Northern Ireland Protoc...,2023-02-01 20:45:45,ie,-0.146
4,646fddfd0576b45037ddb2ca,86433356,Shoppers must demand longer lasting goods and ...,2023-01-10 16:14:04,ie,-0.077


In [29]:
irelandNewsDF.describe()

,id,sentiment
count,5.050000e+02,505.000000
mean,9.049021e+07,-0.003907
std,1.450734e+07,0.288732
min,6.496858e+07,-0.774000
25%,7.718277e+07,-0.216000
50%,9.012608e+07,-0.029000
75%,1.034486e+08,0.186000
max,1.124263e+08,0.879000


In [30]:
irelandNewsDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   _id             505 non-null    object 
 1   id              505 non-null    int64  
 2   title           505 non-null    object 
 3   publish_date    505 non-null    object 
 4   source_country  505 non-null    object 
 5   sentiment       505 non-null    float64
dtypes: float64(1), int64(1), object(4)
memory usage: 23.8+ KB


In [31]:
irelandNewsDF.drop(["_id", "id"], axis=1, inplace=True)
irelandNewsDF["publish_date"] = pd.to_datetime(irelandNewsDF["publish_date"])
irelandNewsDF.head()

,title,publish_date,source_country,sentiment
0,Kilkenny-based recruitment company wins two in...,2023-04-18 08:59:00,ie,0.544
1,Ministers Harris and Ryan welcome record numbe...,2023-01-17 21:41:06,ie,0.341
2,New €4.8m state-of-the-art facility planned fo...,2022-12-05 11:06:50,ie,0.377
3,‘Not possible’ to make Northern Ireland Protoc...,2023-02-01 20:45:45,ie,-0.146
4,Shoppers must demand longer lasting goods and ...,2023-01-10 16:14:04,ie,-0.077


In [47]:
irelandNewsDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           505 non-null    object        
 1   publish_date    505 non-null    datetime64[ns]
 2   source_country  505 non-null    object        
 3   sentiment       505 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 15.9+ KB


In [32]:
# Replace the &amp;rsquo; and &amp;lsquo; with ' symbols
irelandNewsDF['title'] = irelandNewsDF['title'].str.replace('&amp;rsquo;', "'")
irelandNewsDF['title'] = irelandNewsDF['title'].str.replace('&amp;lsquo;', "'")

In [46]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
irelandNewsDF['title'] = irelandNewsDF['title'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
irelandNewsDF['title'].head()

0    Kilkenny-based recruitment company wins two in...
1    Ministers Harris Ryan welcome record number en...
2    New €4.8m state-of-the-art facility planned UC...
3    ‘Not possible’ make Northern Ireland Protocol ...
4    Shoppers must demand longer lasting goods less...
Name: title, dtype: object